In [3]:
!pip install pyspark==3.5.2

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

from transformers import pipeline
import logging
import os

logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - $(message)s')

checkpoint_dir = '/kaggle/working/checkpoints/kafka_to_mongo'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

config = {
    'kafka': {
        "bootstrap.servers": "pkc-l7pr2.ap-south-1.aws.confluent.cloud:9092",
        "security.protocol": "SASL_SSL",
        "sasl.mechanisms": "PLAIN",
        "sasl.username": "kafka-cluster-api-key",
        "sasl.password": "kafka-cluster-api-secret",
        "client.id": "json-serial-producer"
    },
    'mongodb': {
        "uri": "mongodb+srv://<sparkuser>:<password>@yelp-cluster.zbveqrm.mongodb.net/?retryWrites=true&w=majority",
        "database": "reviewsdb",
        "collection": "enriched_reviews_collection"
    }
}

sentiment_pipeline = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

def analyze_sentiment(text):
    if text and isinstance(text, str):
        try:
            result = sentiment_pipeline(text)[0]
            return result['label']
        except Exception as e:
            logging.error(f"Error in sentiment analysis: {e}")
    return "Empty or Invalid"

sentiment_udf = udf(analyze_sentiment, StringType())

def read_from_kafka_and_write_to_mongo(spark):
    topic = "raw_topic"
    schema = StructType([
        StructField("review_id",StringType()),
        StructField("user_id",StringType()),
        StructField("business_id",StringType()),
        StructField("stars",FloatType()),
        StructField("useful",IntegerType()),
        StructField("funny",IntegerType()),
        StructField("cool",IntegerType()),
        StructField("text",StringType()),
        StructField("date",StringType())
    ])

    stream_df = (spark
                     .readStream
                     .format('kafka')
                     .option('kafka.bootstrap.servers', config['kafka']['bootstrap.servers'])
                     .option('subscribe', topic)
                     .option('kafka.security.protocol', config['kafka']['security.protocol'])
                     .option('kafka.sasl.mechanism', config['kafka']['sasl.mechanisms'])
                     .option("kafka.sasl.jaas.config",
                        f'org.apache.kafka.common.security.plain.PlainLoginModule required username="{config["kafka"]["sasl.username"]}" '
                        f'password="{config["kafka"]["sasl.password"]}";')
                 .option("failOnDataLoss","false")
                 .load()
                )
    parsed_df = stream_df.select(from_json(col('value').cast("string"), schema).alias("data")).select("data.*")

    enriched_df = parsed_df.withColumn("sentiment", sentiment_udf(col('text')))
    
    query = (enriched_df.writeStream
             .format("mongodb")
             .option("spark.mongodb.connection.uri", config['mongodb']['uri'])
             .option("spark.mongodb.database", config['mongodb']['database'])
             .option("spark.mongodb.collection", config['mongodb']['collection'])
             .option("checkpointLocation", checkpoint_dir)
             .outputMode("append")
             .start()
             .awaitTermination()
            )

if __name__ == "__main__":
    spark = (SparkSession.builder
          .appName("KafkaStreamToMongo")
          .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.2,org.mongodb.spark:mongo-spark-connector_2.12:10.4.0")
          .getOrCreate()
          )
    read_from_kafka_and_write_to_mongo(spark)



Device set to use cuda:0
25/05/11 15:02:52 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/11 15:02:52 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/11 15:03:00 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/05/11 15:03:00 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/05/11 15:03:00 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/05/11 15:03:02 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/05/11 15:03:02 WARN KafkaDataConsumer: KafkaDataConsumer is not running 